# Introduction to LangChain e Hugging Face  

# Packages

# Settings
 

*Transformers* by HuggingFace offers a wide range of pre-trained models like BERT, GPT, and T5 for NLP tasks. 

*Einops* simplifies tensor manipulation with a clear syntax, making complex operations more straightforward. 

*Accelerate*, also by HuggingFace, helps optimize model training on various hardware accelerators such as GPUs and TPUs. 

*BitsAndBytes* enables efficient quantization of large models, reducing memory consumption in PyTorch.

In [3]:
# !pip install -q transformers einops accelerate bitsandbytes

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

2025-01-09 17:21:35.413551: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-09 17:21:35.413664: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-09 17:21:35.539125: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-09 17:21:35.804266: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-09 17:21:38.011785: W tensorflow/compiler/tf2

In [5]:
import torch
import getpass
import os

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [6]:
device

'cpu'

In [7]:
torch.random.manual_seed(42)

# Token

In [9]:
os.environ["HF_TOKEN"] = getpass.getpass()

# Model
Model from HuggingFace  

Starting by showcasing Phi 3 (microsoft/Phi-3-mini-4k-instruct), a smaller model that has proven to be very interesting and comparable to much larger ones.

https://huggingface.co/microsoft/Phi-3-mini-4k-instruct

open source, accessible, and performs well in Portuguese, although it still works better in English.

In [12]:
id_model = "microsoft/Phi-3-mini-4k-instruct"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    id_model,
    device_map = "cuda",
    torch_dtype = "auto",
    trust_remote_code = True,
    attn_implementation="eager"
)

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.

*device_map="cuda":* Specifies that the model should be loaded onto a CUDA-enabled GPU. GPUs significantly improve inference and training performance by leveraging parallel processing.

*torch_dtype="auto":* Automatically sets the appropriate data type for the model's tensors. This ensures the model uses the best data type for performance and memory efficiency, typically float32 or float16.

*trust_remote_code=True:* Allows the loading of custom code from the model repository on HuggingFace. This is necessary for certain models that require specific configurations or implementations not included in the standard library.

*attn_implementation="eager":* Specifies the implementation method for the attention mechanism. The "eager" setting is a particular implementation that may offer better performance for some models by processing the attention mechanism in a specific way.

## Tokenizer
Ee also need to load the tokenizer associated with the model. The tokenizer is essential for preparing text data into a format that the model can understand.

A tokenizer converts raw text into tokens, which are numerical representations that the model can process. It also converts the model's output tokens back into human-readable text.
Tokenizers handle tasks such as splitting text into words or subwords, adding special tokens, and managing vocabulary mapping.
[more details in the slides]

The tokenizer is a critical component in the NLP pipeline, bridging the gap between raw text and model-ready tokens.

To implement this, we will use the AutoTokenizer.from_pretrained() function, specifying the same tokenizer as the model. This ensures consistency in text processing during both training and inference.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(id_model)

## Creating the Pipeline

Now we will create a pipeline for text generation using the model and tokenizer we loaded earlier. HuggingFace's pipeline function simplifies the process of executing various natural language processing tasks by providing a high-level interface.

A pipeline is an abstraction that simplifies the use of pre-trained models for a variety of NLP tasks. It provides a unified API for different tasks, such as text generation, text classification, translation, and more.

> [More details in the slides]

### Parameters:

- **`"text-generation"`**: Specifies the task the pipeline is set up to perform. In this case, we are configuring a pipeline for text generation. The pipeline will use the model to generate text based on a given prompt.
  
- **`model=model`**: Specifies the pre-trained model the pipeline will use. Here, we are passing the previously loaded model. This model is responsible for generating text based on the input tokens.
  
- **`tokenizer=tokenizer`**: Specifies the tokenizer the pipeline will use. We pass the previously loaded tokenizer to ensure that the input text is correctly tokenized and the output tokens are accurately decoded.

In [ ]:
pipe = pipeline("text-generation", model = model, tokenizer = tokenizer)

## Parameters for Text Generation

To customize the behavior of our text generation pipeline, we can pass a dictionary of arguments to control various aspects of the generation process.

### `max_new_tokens`
This parameter specifies the maximum number of new tokens to be generated in response to the input prompt. It controls the length of the generated text.

- **Example**: Setting `max_new_tokens` to 500 means the model will generate up to 500 tokens beyond the input prompt.

### `return_full_text`
Determines whether to return the full text, including the input prompt, or only the newly generated tokens.

- **Example**: Setting `return_full_text` to `False` means only the newly generated tokens will be returned, excluding the original input prompt. If set to `True`, the returned text will include both the input prompt and the generated continuation.

### `temperature`
Controls the randomness of the text generation process. Lower values make the model's output more deterministic and focused, while higher values increase randomness and creativity.

- **Example**: A `temperature` of `0.1` makes the model's predictions more reliable and less varied, leading to more predictable outputs. A higher `temperature` would result in more diverse and varied text.

### `do_sample`
This parameter enables or disables sampling during text generation. When set to `True`, the model samples tokens based on their probabilities, adding an element of randomness to the output. When set to `False`, the model always selects the token with the highest probability (greedy decoding).

- **Example**: Setting `do_sample` to `True` allows for more diverse and creative text generation. If set to `False`, the output will be more deterministic but potentially less engaging.



In [ ]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.1, # 0.1 até 0.9
    "do_sample": True,
}

## Generating the Output

The following line of code passes the input message and generation arguments to the text generation pipeline:

```python
output = pipe(messages, **generation_args)
```
**generation_args: This unpacks the generation_args dictionary and passes its contents as keyword arguments to the pipeline, customizing the text generation process. This allows fine-tuning of the generation behavior by adjusting parameters such as max_new_tokens, temperature, and more.